In [ ]:
#from pyforest import *
import numpy as np
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver import ChromeOptions, Chrome
import re
import time
from pathlib import Path
import os
gecko = 'C:/Program Files (x86)/Mozilla Firefox/geckodriver.exe'

In [ ]:
options = Options()
profile = webdriver.FirefoxProfile()
path="C:\\Users\\Vikram\\Documents\\ML\\git_local\\Vikramardham\\fda_drugs\\data\\"

profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", path)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain,text/x-csv,text/csv,\
                       application/csv,application/x-csv,\
                       text/csv,text/comma-separated-values,\
                       text/x-comma-separated-values,text/tab-separated-values")
options.headless=True

In [ ]:
%%time
URL = "https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=reportsSearch.process"

driver = webdriver.Firefox(executable_path=gecko, options=options, firefox_profile=profile)
driver.get(URL)

temp_path = Path(path)/'DrugsFDA FDA-Approved Drugs.csv'
if temp_path.is_file():
    temp_path.unlink()
    
for years in np.arange(2020, 2021):
    
    yr=str(years)
    
    for mth in ['January', 'February', 'March', 'April','May', 'June', 
                'July', 'August', 'September', 'October', 'November', 'December']:
        
        (Path(path)/yr/mth).mkdir(parents=True, exist_ok=True)
        if temp_path.is_file():
            temp_path.unlink(missing_ok=True)
            
        month = Select(driver.find_element_by_id('reportSelectMonth'))
        year = Select(driver.find_element_by_id('reportSelectYear'))
        month.select_by_visible_text(mth)
        year.select_by_visible_text(yr)

        search=driver.find_element_by_xpath('//*[@title="Click to Search"]').click()

        csv=driver.find_element_by_xpath('//*[@class="btn btn-default buttons-csv buttons-html5"]').click()

        it=0
        while not temp_path.is_file() and it<5:
            time.sleep(0.2)
            it=it+1

        if temp_path.is_file():
            temp_path.rename(Path(path)/yr/mth/'fda.csv')

driver.quit()

In [ ]:
#handling graceful exit of selenium

In [ ]:
#check if all the files have appropriate dates ... print faulty files
for years in np.arange(1900, 2021):
    #print(years)
    for i, mth in enumerate(['January', 'February', 'March', 'April','May', 'June', 
                'July', 'August', 'September', 'October', 'November', 'December']):
        
        temp_path=Path(path)/str(years)/mth/'fda.csv'
        if temp_path.stat().st_size==0:
            #print(years, mth)
            continue
        else:
            #pass
            data = pd.read_csv(temp_path)
            if data.shape[0]>0:
                chk = pd.to_datetime(data[data.columns[0]])[0]
                if (not chk.year==years) and (not chk.month==i+1):
                    print(chk.year, chk.month)

In [ ]:
class FdaApp():

    '''Class to instantiate and download data'''

    def __init__(self, URL):
        self.url = URL
        self.wait = 0.25
        self.months = ['January', 'February', 'March', 'April',
                      'May', 'June', 'July', 'August',
                      'September', 'October', 'November', 'December']
        self.path = Path('/mnt/c/Users/Vikram/Documents/ \
                             ML/git_local/Vikramardham/fda_drugs/')

    def dowload_data(self, year, month=None):
        
        #options = Options()
        options = Options()
        #options.add_argument('--no-sandbox')
        #options.add_argument('-disable-dev-shm-usage')
        options.headless=True
        
        folder = (self.path / 'data' / f'{year}')
        folder.mkdir(parents=True, exist_ok=True)
        options.add_argument(f'download.default_directory={folder}')
        
        #options.headless = True
        try:
            #driver = webdriver.Firefox(options=options)
            driver = Chrome(options=options)
            print('ABLE to Load browser')
        except: 
            print('FAILED: loading options')
            driver.quit()
            return
        try:
            driver.get(self.url)
        except:
            print('FAILED: getting URL')
            driver.quit()
            return 
        
        if month: self.months = [month]
        
        try:
            drop_down_month = Select(driver.find_element_by_id("reportSelectMonth"))
            drop_down_year = Select(driver.find_element_by_id("reportSelectYear"))
        except :
            print('FAILED: @ finding drop down')
            driver.quit()
            return
        
        for mnt in self.months:
            try:
                drop_down_month.select_by_visible_text(mnt)
                drop_down_year.select_by_visible_text(year)
            except:
                print('FAILED: @ checking drop down')
                driver.quit()
                return 
            try:
                search = driver.find_element_by_xpath('//*[@title="Click to Search"]').click()
            except:
                print('FAILED: @ finding search button')
                driver.quit()
                return
            try:
                csv = driver.find_element_by_xpath('//*[@class = "btn btn-default buttons-csv buttons-html5"]').click()
                print('SUCCESS: {month}, {year}')
                driver.quit()
            except:
                print('FAILED: downloading CSV')
                driver.quit()
                return
        #try: 
        #    select = Select(driver.find_element_by_id('ResultsPerPageOptions'))
        #except:
        #    return None
        
        
        return None

In [ ]:
URL = "https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=reportsSearch.process"
#fda = FdaApp('https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=reportsSearch.process')

In [ ]:
#fda.dowload_data(2020, 'March')